# Aplicação da ResNet 50

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [2]:
# Definir as transformações para os dados de treinamento, validação e teste
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [3]:
# Caminhos para os dados de treinamento, validação e teste
train_dir = 'PokemonSplit/train'
val_dir = 'PokemonSplit/val'
test_dir = 'PokemonSplit/test'
data_dirs = {'train': train_dir, 'val': val_dir, 'test': test_dir}

In [4]:
image_datasets = {x: datasets.ImageFolder(data_dirs[x], data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

In [5]:
# Carregar o modelo ResNet50
model_ft = models.resnet50(weights='DEFAULT')

# Congelar todas as camadas exceto a última camada totalmente conectada
for param in model_ft.parameters():
    param.requires_grad = False

# Substituir a última camada totalmente conectada por uma que tenha o número de classes desejado
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

# Mover o modelo para a GPU, se disponível
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

# Definir o otimizador e a função de perda
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [7]:
# Função para treinar o modelo
def train_model(model, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        print('-' * 10)

    return model

# Treinar o modelo
model_ft = train_model(model_ft, criterion, optimizer, num_epochs=25)

Epoch 1/25
train Loss: 3.9540 Acc: 0.2914
val Loss: 2.9915 Acc: 0.5789
----------
Epoch 2/25


KeyboardInterrupt: 

In [ ]:
# Função para testar o modelo
def test_model(model, dataloader):
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    accuracy = running_corrects.double() / len(dataloader.dataset)
    print(f'Test Accuracy: {accuracy:.4f}')

# Testar o modelo
test_model(model_ft, dataloaders['test'])

In [ ]:
# Salvar o modelo treinado
torch.save(model_ft.state_dict(), 'modelo_resnet50.pth')

treinamento e teste processados pela GPU do google colab: https://colab.research.google.com/drive/1juJgxvQIotooA7CyI12svBFZDlaIwVVf?usp=sharing

resultados:
16 minutos treinando e validando
Test Accuracy: 84.15

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Função para mostrar uma imagem
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause to update plots

# Selecionar uma imagem aleatória do conjunto de teste
def get_random_test_image(dataloader):
    images, labels = next(iter(dataloader))
    idx = random.randint(0, len(images) - 1)
    return images[idx], labels[idx]

# Testar o modelo com uma imagem de teste
def test_random_image(model, dataloader, class_names):
    model.eval()
    with torch.no_grad():
        image, label = get_random_test_image(dataloader)
        input_image = image.unsqueeze(0).to(device)
        label = label.to(device)

        outputs = model(input_image)
        _, preds = torch.max(outputs, 1)

        imshow(image.cpu(), title=f'Identificou: {class_names[preds[0]]} (Pokemon: {class_names[label]})')
        if preds[0] == label:
            print("Identificação está correta!")
        else:
            print("Identificação está incorreta.")

In [ ]:
# Testar o modelo com uma imagem aleatória do conjunto de teste
test_random_image(model_ft, dataloaders['test'], class_names)